<a href="https://colab.research.google.com/github/Eric-Musoso/GeoAI/blob/main/Assignment_PA3b_Introduction_to_GEE_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing all necessary libaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import ee
import geemap

## Authentication and Initialization GEE

In [3]:
ee.Authenticate()
ee.Initialize(project='ee-my-ericm')


# **Assignment Section**

## Mathematical Operations on Images

Classifying elevation data into to 2 classes. (class 1: > 1000m, class 2: <=1000m).

In [4]:
dem = ee.Image('CGIAR/SRTM90_V4')
elevation = dem.select('elevation')

elevationMask = elevation.gt(1000)

Visualizing the results

In [5]:
Map = geemap.Map(center = [15.8700,100.9925], zoom = 6)
vis_Para = {'min': 0, 'max': 2585, 'palette': ['brown', 'green']}
Map.addLayer(elevationMask, vis_Para, name="DEM Mask")
Map

Map(center=[15.87, 100.9925], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [6]:
LS8_img = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_127051_20200120')

Map = geemap.Map(center = [12.9674,104.0529], zoom = 8)
vis_Para = {'min': 0, 'max': 30000, 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}
Map.addLayer(LS8_img, vis_Para, name="Exaple LANDSAT Image")

In [7]:
Map

Map(center=[12.9674, 104.0529], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Exercise section

### **Exercise1:**
Play around with threshold to find out best threshold that separate water and land
classes.


In [8]:
LS8_img = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_127051_20200120')

ndvi = LS8_img.expression('(NIR-RED)/(NIR+RED)', {'NIR': LS8_img.select('SR_B5'), 'RED': LS8_img.select('SR_B4')})

Map = geemap.Map(center = [12.9674,104.0529], zoom = 8)
vis_Para = {'min': 0, 'max': 1, 'palette': ['yellow', 'green']}
Map.addLayer(ndvi, vis_Para, name="Tonlesap Lake - NDVI")
# Threshold value
threshold = 0.02

# Apply thresholding to classify water and land
water_land_mask = ndvi.lt(threshold)

# Visualize the water-land mask
Map.addLayer(water_land_mask.updateMask(water_land_mask), {'palette': 'blue'}, '')

# Display the Map
Map

Map(center=[12.9674, 104.0529], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

### **Exercise2:**
 Apply threshold on NDVI image to separate high vegetation area and low
vegetation area and visualize it.

In [9]:
LS8_img = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_127051_20200120')

ndvi = LS8_img.expression('(NIR-RED)/(NIR+RED)', {'NIR': LS8_img.select('SR_B5'), 'RED': LS8_img.select('SR_B4')})

Map = geemap.Map(center = [12.9674,104.0529], zoom = 8)
vis_Para = {'min': 0, 'max': 1, 'palette': ['yellow', 'green']}
Map.addLayer(ndvi, vis_Para, name="Tonlesap Lake - NDVI")
Map

# Threshold value for separating high and low vegetation
threshold = 0.3

# Apply thresholding to classify high and low vegetation
high_veg_mask = ndvi.gt(threshold)

# Visualize the high vegetation mask
Map.addLayer(high_veg_mask.updateMask(high_veg_mask), {'palette': 'green'}, 'High Vegetation')

# Display the Map
Map

Map(center=[12.9674, 104.0529], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

### **Exercise3:**
 Similarly try to calculate another advance vegetation index knows as Enhanced
Vegetation Index (EVI)

In [10]:
# Calculate EVI
def calculate_evi(image):
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    return evi

evi = calculate_evi(LS8_img)

# Create a Map
Map = geemap.Map(center=[12.9674, 104.0529], zoom=8)

# Define visualization parameters
vis_Para = {'min': -1, 'max': 1, 'palette': ['brown', 'yellow', 'green']}

# Add EVI layer to the map
Map.addLayer(evi, vis_Para, name="EVI")

# Display the Map
Map

Map(center=[12.9674, 104.0529], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

### **Exercise4:**
 Edit above code to calculate square of NDVI difference and visualize it

In [11]:
# Load the Landsat images for 1999 and 2008
ls1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
ls2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Calculate NDVI for each time period
ls1999_ndvi = ls1999.expression('(NIR-RED)/(NIR+RED)', {'NIR': ls1999.select('B4'), 'RED': ls1999.select('B3')})
ls2008_ndvi = ls2008.expression('(NIR-RED)/(NIR+RED)', {'NIR': ls2008.select('B4'), 'RED': ls2008.select('B3')})

# Calculate the difference in NDVI between 2008 and 1999
ndvi_diff = ls2008_ndvi.subtract(ls1999_ndvi)

# Square the NDVI difference
ndvi_diff_squared = ndvi_diff.multiply(ndvi_diff)

# Create a Map
Map = geemap.Map(center=[15.8700, 100.9925], zoom=6)

# Define visualization parameters
vis_Para = {'min': 0, 'max': 0.25, 'palette': ['brown', 'green']}

# Add squared NDVI difference layer to the map
Map.addLayer(ndvi_diff_squared, vis_Para, name="Squared NDVI Diff 2008 - 1999")

# Display the Map
Map

Map(center=[15.87, 100.9925], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…